In [33]:
# Импортируем необходимые объекты
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import time
import random
import json
import re
import pandas as pd
import sqlite3
from sqlite3 import Error
import os 
import datetime

In [34]:
# ------- Этот блок содержит глобальные переменные, необходимые для работы --------- #

#Далее создается элемент класса Firefox WebDriver для нашего url
#Единый указатель ресурса (англ. Uniform Resource Locator, URL 
# — единообразный локатор (определитель местонахождения) ресурса (в нашем случае сайт Картотеки арбитражных дел)
url = 'http://ras.arbitr.ru/'  
path_to_user_driver = 'C:/Users/besso/Desktop/VKR/chromedriver.exe'
path_to_anticaptcha_plugin = 'C:/Users/besso/Desktop/VKR/anticaptcha-plugin_v0.3410.zip' # Имеется в виду Ссылка на CRX или ZIP или ZPI файл плагина, который мы скачали
anti_captcha_key = '3d8da2d26ff3ba9552e5524629cface8'



In [35]:
#Параметры, которые должны быть забиты из приложения !

date_from = ''
date_to = ''
doc_text_input = 'казенное учреждение'
download_directory = 'C:\\Users\\besso\\Desktop\\VKR\\'

#собственно сюда нужно забить необходимые параметры

#C:\Users\besso\Desktop\VKR

In [36]:
# Метод отправки API запроса прямо в плагин
# Например для инициализации API ключа сервиса anti-captcha.com, необходимый для работы плагина
# Работает только на действующей HTML страничке,
# в нашем случае на https://antcpt.com/blank.html
# на страницах вроде about:blank запрос не пройдет
def acp_api_send_request(driver, message_type, data={}):
    message = {
        # всегда указывается именно этот получатель API сообщения
        'receiver': 'antiCaptchaPlugin',
        # тип запроса, например setOptions
        'type': message_type,
        # мерджим с дополнительными данными
        **data
    }
    # выполняем JS код на странице
    # а именно отправляем сообщение стандартным методом window.postMessage
    return driver.execute_script("""
    return window.postMessage({});
    """.format(json.dumps(message)))


In [37]:
def init_chrome_options(path_to_anticaptcha_plugin):
    # Инциируем объект опций для Хрома, чтобы иметь возможность подключить расширение
    options = webdriver.ChromeOptions()
    # Ссылка на CRX или ZIP или ZPI файл плагина, который мы скачали ранее
    options.add_extension(path_to_anticaptcha_plugin)
    prefs = {"plugins.always_open_pdf_externally": True, 
            'download.default_directory': download_directory} #Без дополнительных прожатий кнопки "скачать" при открытии pdf-скачиваем его
    options.add_experimental_option("prefs",prefs)
    return options

In [38]:
# Ф-ция для инцииализации веб-драйвера хрома опциями
def get_browser_with_options(path_to_user_driver):
    # Запускаем Браузер (Веб Драйвер Хрома) с указанием места скачивания самого файла драйвера
    browser = webdriver.Chrome(path_to_user_driver, options=init_chrome_options(path_to_anticaptcha_plugin) )
    return browser

In [39]:
# Ф-ция для получения стартовой страницы и обхода капчи на ней
def get_initial_page(  url = url, anti_captcha_key = anti_captcha_key  ):
    # Переходим на пустую страницу для выполнения API запроса к плагину
    browser = get_browser_with_options(path_to_user_driver)
    
    browser.get(url)
    
    #здесь делаем закрытие объявления (в случае, если оно появилось)
    #если объявление на сайте не появилось, то Selenium просто не найдет объект, отвечающий за него,
    #и при попытке закрыть объявление сгенерируется исключение
    #чтобы из-за этого исключения не падал весь дальнейший код, делаем обработку исключения с возможностью
    #выполнения остатка кода
    
    try:
        button_close = browser.find_element_by_class_name('b-promo_notification-popup-close')
        button_close.click()
    except NoSuchElementException:
        pass
    
    # Устанавливаем API ключ anti-captcha.com
    # замените YOUR-ANTI-CAPTCHA-API-KEY на Ваш шестнадцатиричный ключ, который можно взять тут:
    # https://anti-captcha.com/clients/settings/apisetup
    
    acp_api_send_request(
        browser,
        'setOptions',
        {'options': {'antiCaptchaApiKey': anti_captcha_key}}
    )

    # Три секунды паузы чтобы плагин проверил ключ на стороне anti-captcha.com
    time.sleep(10)
    
    return browser



In [11]:
# Необходимо добавить функцию, которая будет колбасить курсором рандомно во избежание бана

In [40]:
def human_text_enter( element, text_input ):
    time.sleep(random.uniform(2, 5)) 
    # sending_text = 'мошенничество'  text input 
    #document_text = browser.find_element_by_xpath('//textarea[@placeholder="текст документа"]') 
    for letter in text_input: 
        time.sleep(random.uniform(0.05, 0.16)) 
        element.send_keys(letter)
    

In [41]:
# Ф-ция, которая забивает форму на сайте нужными нам параметрами (пока параметры не все, потом добавим, сейчас в тестовом режиме)
def get_page_by_input_attributes(browser= get_initial_page(  url = url, anti_captcha_key = anti_captcha_key  ), doc_text_in = '', date_from_in = '', date_to_in = '' ):
    
    document_text = browser.find_element_by_xpath('//textarea[@placeholder="текст документа"]') #текст документа
   # document_text.send_keys(doc_text_in)
    
    human_text_enter(document_text, doc_text_in) # Эмулируем ввод текста документа человеком
    
    #dispute_type = browser.find_element_by_xpath('//input[@placeholder="вид спора"]') #вид спора
    #dispute_category = browser.find_element_by_xpath('//input[@placeholder="категория спора"]') #категория спора
    
    #case_participant = browser.find_element_by_xpath('//textarea[@placeholder="название, ИНН или ОГРН"]') #название, ИНН или ОГРН
    #court = browser.find_element_by_xpath('//input[@placeholder="название суда"]') #название суда
    #court.send_keys(court_in)
    #case_number = browser.find_element_by_xpath('//input[@placeholder="например, А50-5568/08"]') #номер дела
    
    date_from = browser.find_element_by_xpath('//label[@class="from"]//input[@placeholder="дд.мм.гггг"]') #нижняя граница даты
    human_text_enter(date_from, date_from_in) # Эмулируем ввод дат человеком
    
    date_to = browser.find_element_by_xpath('//label[@class="to"]//input[@placeholder="дд.мм.гггг"]') #верхняя граница даты
    human_text_enter(date_to, date_to_in)
    
    search_button = browser.find_element_by_xpath('//button[@alt="Найти"]') #кнопка "Найти"
    
    # Самая важная чаcть: ждем не более 120 секунд пока индикатор антикаптчи с классом antigate_solver
    # не получит класс solved, что означает что рекапча решена
    WebDriverWait(browser, 120)
    
    # Отправляем форму
    search_button.click()
    
    time.sleep(10)
    #ищем ссылку для перехода на страницу документа и переходим по ней
    return browser


In [42]:
initial_parsing_page = get_page_by_input_attributes(doc_text_in =  doc_text_input ) # пока это текущий браузер

In [43]:
def get_row_links_from_1_page(fist_page=initial_parsing_page):
    links = initial_parsing_page.find_elements_by_xpath("//a[@href]")
    all_links = list()
    for link in links:
        all_links.append(link.get_attribute("href"))
    return all_links
   

In [44]:
def get_pdf_links(all_links): 
    pdf_links = list()
    for link in all_links:
        if '.pdf' in link:
            pdf_links.append(link)
        else:
            continue
    return pdf_links


In [45]:
def count_found_documents(intial_page = initial_parsing_page):
    total_count = initial_parsing_page.find_elements_by_xpath('//*[@id="contentHeader"]/span[1]')
    split_text = total_count[0].text.split()
    return int(split_text[1])

In [46]:
def current_doc_info_text(intial_page, n  ):
    pattern_searh = '//*[@id="b-cases"]/li['
    pattern_searh = pattern_searh + str(n) + ']'
    try:
        current_page = initial_parsing_page.find_elements_by_xpath(pattern_searh) # будем итерироваться по индексу
        description = current_page[0].text
    except:
        NoSuchElementException
        pass
    return description

#можем выцепить порядковый номер дела, Номер дела, Инстанция, дата размещения документа , название документа

In [48]:
# Получаем данные для заполнения бд 
def get_doc_attributes(description ):
    ordinal_doc_number = int(description[0 : description.find('.') ]) # порядковый номер дела (нужен для показа загруженных материалов)
    start_pos_case_number = description.find('\n')
    end_pos_case_number = description.find('\n', start_pos_case_number + 1 ) # ищет второе вхождение этого символа в строке
    case_number = description[start_pos_case_number + 1 : end_pos_case_number ] # Номер дела
    try:
        date_doc = re.findall("\d{2}.\d{2}.\d{2,4}", description)[1] # дата заливки документа
    except:
        IndexError # индекс не входит в диапазон элементов.
        date_doc = re.findall("\d{2}.\d{2}.\d{2,4}", description)[0] # дата заливки документа
    
    court_name = description[end_pos_case_number + 1 : description.find(date_doc)] # Название суда, выносившего решение

    start_doc_pos = description.find(date_doc) + 10
    doc_name = description[ start_doc_pos : ]
    indexs = [i for i, symb in enumerate(doc_name) if symb=='\n']
    document_name = '' # название документа
    for i in range(len(doc_name)):
        if i not in indexs:
            document_name +=  doc_name[i] 
    
    return ordinal_doc_number, case_number, date_doc, court_name, document_name

    

In [49]:
# Скачиваются все файлы с текущей страницы браузера. Доработать так, чтобы итерировалось еще по другим страницам
def download_description(browser, pdf_links ):
    
    insert_values = { 'ordinal_number' : [], 
                      'case_number' : [],
                      'date_doc' : [], 
                      'court_name' : [],
                      'document_name' : [],
                      'pdf_link' : [],
                    }
    
    cnt_documents = count_found_documents() # Общее количество документов
    
    if cnt_documents > 40 * 25:
        cnt_documents = 40 * 25 # Сайт выдает только 40 страниц по 25 документов 
    
    for i in range(len(pdf_links)):
        
        description = current_doc_info_text(browser, i + 1 ) # получили описание нужного дока 
        
        ordinal_doc_number, case_number, date_doc, court_name, document_name = get_doc_attributes(description = description)
        
        
        # добавить в словарь key - value и расширять его по ключам 
        
        
        print("Идет загрузка документа {0} из {1}".format(ordinal_doc_number, cnt_documents))
        print(ordinal_doc_number)
        if pdf_links[i] not in insert_values['pdf_link']:
            insert_values['ordinal_number'].append(ordinal_doc_number)
            print(case_number)
            insert_values['case_number'].append(case_number)
            print(date_doc)
            insert_values['date_doc'].append(date_doc)
            print(court_name)
            insert_values['court_name'].append(court_name)
            print(document_name)
            insert_values['document_name'].append(document_name)
            print(pdf_links[i]) # просто линк текущего документа
            insert_values['pdf_link'].append(pdf_links[i])
        #time.sleep(random.uniform(2, 3))  - раскомментить
        else:
            continue
    return insert_values
       
    
    
   

In [64]:
#browser_after_1_page = download_pages(browser=initial_parsing_page,pdf_links=pdf_links) # Осторожно. Файлы начнут скачиваться подряд и все!

In [50]:
def parse_all_documents(initial_parsing_page):
    cnt_documents = count_found_documents(initial_parsing_page) # Общее количество документов
    # Проход по всем страницам 
    flag = True
    i = 1 # для теста нужно заменить, чтобы было не с 1 страницы 
    x_path = '//*[@id="pages"]/li'
    current_page = initial_parsing_page
    links_need_to_be_downloaded = list()
    list_of_insert_values = list()
    while flag and i <= 12 : # убрать вторую часть условия
            print(i)
            if i < 10:
                try :
                    #print("page nom ", i)
                    next_page = initial_parsing_page.find_element_by_xpath( x_path + '[' + str(i + 2) + ']/a') # После парсинга текущей страницы переходим к следующей
                    #print(x_path + '[' + str(i + 1) + ']/a')
                   # print("ya tut 1 ")
                    raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page) # Получаем все "сырые ссылки с текущей страницы"
                    #print("get_row_links_from_1_page = success")
                    pdf_links = get_pdf_links(raw_links) # Получаем ссылки на pdf-доки
                    #print("get_pdf_links = success")
                    links_need_to_be_downloaded += pdf_links
                    
                    insert_values = download_description(initial_parsing_page, pdf_links)
                    list_of_insert_values.append(insert_values)
                    #download_docs(browser, pdf_links)
                    
                    next_next_page = next_page.click()
                    #print(x_path + '[' + str(i + 1) + ']/a')
                    i += 1 
                    time.sleep(random.uniform(5, 10))         
                except : 
                    NoSuchElementException
                    flag = False                    
                    
            elif i % 10 == 0:
                try:
                    if i == 10: 
                        j = 11
                    else :
                        j = 13 

                
                    #print(x_path + '[' + str(i + 1) + ']/a' + 'elif')
                    next_page = initial_parsing_page.find_element_by_xpath( x_path + '[' + str(j) + ']/a') # После парсинга текущей страницы переходим к следующей
                
                    
                    raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page) # Получаем все "сырые ссылки с текущей страницы"
                    #print("get_row_links_from_1_page- succ")
                    pdf_links = get_pdf_links(raw_links) # Получаем ссылки на pdf-доки
                    #print("get_pdf_links - succ ")
                    links_need_to_be_downloaded += pdf_links
                    
                    insert_values = download_description(initial_parsing_page, pdf_links)
                    #print("download_description - succ ")
                    list_of_insert_values.append(insert_values) 
                    #print("append - succ ")
                    
                    #download_docs(browser, pdf_links)
                    
                    next_next_page = next_page.click()
                    #print(x_path + '[' + str(j) + ']/a')
                    i += 1 
                    time.sleep(random.uniform(5, 10))          
                    next_page = initial_parsing_page.find_element_by_xpath(x_path + '[' + str(j + 1 ) + ']/a') # После парсинга текущей страницы переходим к следующей
                    next_page.click()
                    time.sleep(random.uniform(5, 10))  

                except:
                    NoSuchElementException
                    flag = False
            else:
                try:
                    next_page = initial_parsing_page.find_element_by_xpath( x_path + '[' + str(i % 10  + 3)  + ']/a') # После парсинга текущей страницы переходим к следующей
                    #print("np=", next_page)
                    #print("ya tut ")
                    raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page) # Получаем все "сырые ссылки с текущей страницы"
                    #print("get_row_links_from_1_page - else block  ")
                    pdf_links = get_pdf_links(raw_links) # Получаем ссылки на pdf-доки
                    #print("get_pdf_links - else block  ")
                    links_need_to_be_downloaded += pdf_links
                    
                    insert_values = download_description(initial_parsing_page, pdf_links)
                    #print("download_description - else block  ")
                    
                    # добавить проверку на то, что если есть такая ссылка, то не добавлять ее в список!
                    
                    
                    list_of_insert_values.append(insert_values)
                    #print("append - else block  ")
                    #download_docs(browser, pdf_links)
                    
                    next_page.click()
                    #print("click - else block  ")
                    i += 1 
                    time.sleep(random.uniform(5, 10))        
                except : 
                    NoSuchElementException
                    flag = False
    # Если мы здесь - значит следующую страницу найти нельзя ( значит надо обработать текущую, т.е. последннюю )
    raw_links = get_row_links_from_1_page(fist_page=initial_parsing_page) # Получаем все "сырые ссылки с текущей страницы"
    pdf_links = get_pdf_links(raw_links) # Получаем ссылки на pdf-доки
    links_need_to_be_downloaded += pdf_links       
    insert_values = download_description(initial_parsing_page, pdf_links)
    list_of_insert_values.append(insert_values)
   # initial_parsing_page.quit() -- раскомментить
                    
                    
          
        
    return links_need_to_be_downloaded, list_of_insert_values
                
                
                

In [ ]:
all_needed_links, list_of_insert_values = parse_all_documents(initial_parsing_page=initial_parsing_page)
# посмотреть до точки и дальше в словарь оформить нужные столбцы
# спроектировать бд

1
Идет загрузка документа 1 из 1000
1
А14-18572/2019
24.12.2020
АС Воронежской области 
РЕШЕНИЕ СУДА ПЕРВОЙ ИНСТАНЦИИИСК УДОВЛЕТВОРИТЬ ЧАСТИЧНО
http://ras.arbitr.ru/Kad/PdfDocument/56716b9d-e758-4823-82ca-ae8f08b396aa/f9b0f78a-0bac-4e11-b166-9660e60737bc/%D0%9014-18572-2019__20201224.pdf
Идет загрузка документа 2 из 1000
2
А12-1479/2020
02.04.2020
АС Волгоградской области 
РЕЗОЛЮТИВНАЯ ЧАСТЬ РЕШЕНИЯ СУДА ПО ДЕЛУ, РАССМАТРИВАЕМОМУ В ПОРЯДКЕ УПРОЩЕННОГО ПРОИЗВОДСТВАИСК УДОВЛЕТВОРИТЬ ПОЛНОСТЬЮ
http://ras.arbitr.ru/Kad/PdfDocument/e078f667-5af2-453a-a7e2-f28b27f332f1/df3e6bc4-88b9-4aeb-93d1-37b670b88a83/%D0%9012-1479-2020__20200402.pdf
Идет загрузка документа 3 из 1000
3
А12-6259/2020
02.04.2020
АС Волгоградской области 
О НЕВОЗМОЖНОСТИ ИСПОЛНЕНИЯ СУДЕБНОГО ПОРУЧЕНИЯ
http://ras.arbitr.ru/Kad/PdfDocument/ce137e42-d23e-4f51-8a24-bfd5e6324a26/3cab4b77-e00a-40f1-b892-aeca19c50c09/%D0%9012-6259-2020__20200402.pdf
Идет загрузка документа 4 из 1000
4
А12-8084/2020
02.04.2020
АС Волгоградской обла

Идет загрузка документа 32 из 1000
32
А55-195/2020
01.04.2020
АС Самарской области 
[ПОДПИСАНО]ПРИНЯТЬ К РАССМОТРЕНИЮ ТРЕБОВАНИЕ КРЕДИТОРАНАЗНАЧИТЬ СУДЕБНОЕ ЗАСЕДАНИЕ ПО РАССМОТРЕНИЮ ТРЕБОВАНИЯ КРЕДИТОРА (СТ.71, СТ.100 ФЗ О НЕСОСТОЯТЕЛЬНОСТИ)
http://ras.arbitr.ru/Kad/PdfDocument/bb07e4ec-abb4-4e68-8d42-37cf2b181e95/7c3695a7-b02e-4db2-b595-39bb26e504d0/%D0%9055-195-2020__20200401.pdf
Идет загрузка документа 33 из 1000
33
А56-130510/2019
01.04.2020
АС города Санкт-Петербурга и Ленинградской области 
[ПОДПИСАНО]УДОВЛЕТВОРИТЬ ИСК ПОЛНОСТЬЮ ИЛИ ЧАСТИЧНО
http://ras.arbitr.ru/Kad/PdfDocument/86034067-0a57-4660-a1d6-229024491f21/64aa9c7e-f9e6-47c6-8836-11844402e1c2/%D0%9056-130510-2019__20200401.pdf
Идет загрузка документа 34 из 1000
34
А12-8088/2020
01.04.2020
АС Волгоградской области 
СУДЕБНЫЙ ПРИКАЗИСК УДОВЛЕТВОРИТЬ ПОЛНОСТЬЮ
http://ras.arbitr.ru/Kad/PdfDocument/74ebf400-84ac-4e24-8112-58c7d93e0f4e/7eb4198c-b9c0-4488-a13d-0f888608b5ec/%D0%9012-8088-2020__20200401.pdf
Идет загрузка документа

Идет загрузка документа 64 из 1000
64
А55-195/2020
01.04.2020
АС Самарской области 
[ПОДПИСАНО]ИЗМЕНИТЬ ДАТУ СУДЕБНОГО ЗАСЕДАНИЯ ПО РАССМОТРЕНИЮ ТРЕБОВАНИЯ КРЕДИТОРА (ПО АНАЛОГИИ СО СТ.158 АПК, СТ.71, СТ.100 ФЗ О НЕСОСТОЯТЕЛЬНОСТИ)ОТЛОЖИТЬ СУДЕБНОЕ ЗАСЕДАНИЕ ПО РАССМОТРЕНИЮ ТРЕБОВАНИЯ КРЕДИТОРА (СТ.158 АПК, СТ.71, СТ.100 ФЗ О НЕСОСТОЯТЕЛЬНОСТИ)
http://ras.arbitr.ru/Kad/PdfDocument/bb07e4ec-abb4-4e68-8d42-37cf2b181e95/7e411279-139f-4a84-9718-d2d8b80c85d8/%D0%9055-195-2020__20200401.pdf
Идет загрузка документа 65 из 1000
65
А32-56232/2019
01.04.2020
15 арбитражный апелляционный суд 
[ПОДПИСАНО]ОСТАВИТЬ БЕЗ ИЗМЕНЕНИЯ РЕШЕНИЕ, А АПЕЛЛЯЦИОННУЮ ЖАЛОБУ - БЕЗ УДОВЛЕТВОРЕНИЯ (П.1 СТ.269 АПК)
http://ras.arbitr.ru/Kad/PdfDocument/b0512f02-9499-4524-9fd4-5c5f949e4a03/9b34677f-8dae-4fb3-9ccb-22b4b41c70d9/%D0%9032-56232-2019__20200401.pdf
Идет загрузка документа 66 из 1000
66
А12-5643/2020
01.04.2020
АС Волгоградской области 
О ВОЗВРАЩЕНИИ ИСКОВОГО ЗАЯВЛЕНИЯ (ЗАЯВЛЕНИЯ)
http://ras.arbitr.ru/Kad/PdfD

Идет загрузка документа 95 из 1000
95
А66-7935/2019
31.03.2020
АС Тверской области 
ВКЛЮЧИТЬ ТРЕБОВАНИЯ В РЕЕСТР ТРЕБОВАНИЙ КРЕДИТОРОВ (СТ.60, 71, 81, 100, 142 ФЗ О НЕСОСТОЯТЕЛЬНОСТИ)
http://ras.arbitr.ru/Kad/PdfDocument/b25788d9-34da-4c0d-ad6c-03084724ff5f/c2f2d28d-8d87-4854-9f54-41a8fc7a705c/%D0%9066-7935-2019__20200331.pdf
Идет загрузка документа 96 из 1000
96
А03-2833/2020
31.03.2020
АС Алтайского края 
ОБ ОТМЕНЕ СУДЕБНОГО ПРИКАЗА
http://ras.arbitr.ru/Kad/PdfDocument/065e6211-cdfd-40a5-b8eb-e2e1358c88c7/8de93c29-8220-4304-bdd8-8b6fc01e8623/%D0%9003-2833-2020__20200331.pdf
Идет загрузка документа 97 из 1000
97
А66-601/2020
31.03.2020
АС Тверской области 
ОТКАЗАТЬ ВО ВЗЫСКАНИИ ОБЯЗАТЕЛЬНЫХ ПЛАТЕЖЕЙ И САНКЦИЙ (СТ.216 АПК РФ)
http://ras.arbitr.ru/Kad/PdfDocument/d4cd46f6-9c1d-4a9d-bc42-c9bb5c983de1/00f1a943-50ee-4667-8b72-6deac7cffea9/%D0%9066-601-2020__20200331.pdf
Идет загрузка документа 98 из 1000
98
А12-7243/2020
31.03.2020
АС Волгоградской области 
СУДЕБНЫЙ ПРИКАЗИСК УДОВЛЕТВОРИТЬ

In [105]:
res_insert_values = pd.concat([pd.DataFrame(d) for d in list_of_insert_values]).to_dict('list')

# объединить словари  в один большой для записи в таблицы

In [90]:
print(res_insert_values)

{'ordinal_number': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218

In [106]:
def data_frame(res_insert_values):
    return pd.DataFrame.from_dict(res_insert_values)

In [107]:
df = data_frame(res_insert_values)


In [108]:
df

,ordinal_number,case_number,date_doc,court_name,document_name,pdf_link
0,1,А14-18572/2019,24.12.2020,АС Воронежской области,РЕШЕНИЕ СУДА ПЕРВОЙ ИНСТАНЦИИИСК УДОВЛЕТВОРИТЬ...,http://ras.arbitr.ru/Kad/PdfDocument/56716b9d-...
1,2,А63-22992/2019,26.03.2020,АС Ставропольского края,"МОТИВИРОВАННОЕ РЕШЕНИЕ ПО ДЕЛУ, РАССМОТРЕННОМУ...",http://ras.arbitr.ru/Kad/PdfDocument/75503dc0-...
2,3,А40-244066/2019,22.03.2020,9 арбитражный апелляционный суд,"ОСТАВИТЬ БЕЗ ИЗМЕНЕНИЯ РЕШЕНИЕ, А АПЕЛЛЯЦИОННУ...",http://ras.arbitr.ru/Kad/PdfDocument/efe9120e-...
3,4,А32-46983/2017,22.03.2020,15 арбитражный апелляционный суд,"ОСТАВИТЬ БЕЗ ИЗМЕНЕНИЯ РЕШЕНИЕ, А АПЕЛЛЯЦИОННУ...",http://ras.arbitr.ru/Kad/PdfDocument/e273ea35-...
4,5,А26-12437/2019,22.03.2020,АС Республики Карелия,[ПОДПИСАНО]ОТКАЗАТЬ В ИСКЕ,http://ras.arbitr.ru/Kad/PdfDocument/029dd9c9-...
...,...,...,...,...,...,...
320,296,А27-712/2020,19.03.2020,АС Кемеровской области,"РЕЗОЛЮТИВНАЯ ЧАСТЬ РЕШЕНИЯ СУДА ПО ДЕЛУ, РАССМ...",http://ras.arbitr.ru/Kad/PdfDocument/dca02904-...
321,297,А10-4121/2018,19.03.2020,АС Республики Бурятия,ОБ ИСТРЕБОВАНИИ ДОКАЗАТЕЛЬСТВ,http://ras.arbitr.ru/Kad/PdfDocument/3154ad8a-...
322,298,А60-58923/2019,19.03.2020,АС Свердловской области,[ПОДПИСАНО]О ПРИНЯТИИ ТРЕБОВАНИЙ КРЕДИТОРА К Р...,http://ras.arbitr.ru/Kad/PdfDocument/9775a4ce-...
323,299,А12-5538/2020,19.03.2020,АС Волгоградской области,О ПРИНЯТИИ ЗАЯВЛЕНИЯ О ПРИЗНАНИИ ДОЛЖНИКА БАНК...,http://ras.arbitr.ru/Kad/PdfDocument/81e5e212-...


In [ ]:
# Создание бд!
#почистить дистинктом дубли

In [109]:
name_db = 'my_vkr.db'

In [110]:
def sql_connection(db_name = name_db):
    try:
        con = sqlite3.connect(name_db) # устанавливаем соединение с БД
        return con
    except Error:
        print(Error)
        

In [111]:
con = sql_connection()

In [112]:
def create_tables(connection = con):
    cursor_obj = con.cursor()
    cursor_obj.execute(""" CREATE TABLE SEARCH_PARAMS(
                           ID_DOC INTEGER,
                           SEARCH_DIRECTION TEXT,
                           DATE_FROM TEXT, 
                           DATE_TO TEXT,
                           DOWNLOAD_DATE TEXT,
                           LOCATED_IN_DIRECTORY TEXT ) """)
    con.commit()
    
    cursor_obj = con.cursor()
    cursor_obj.execute(""" CREATE TABLE DOC_INFO(
                           ID_DOC INTEGER, 
                           CASE_NUMBER TEXT, 
                           DATE_ON_SITE TEXT, -- LAST DATE OF UPDATING DOCUMENT ON SITE 
                           COURT_NAME TEXT, 
                           DOC_TITLE TEXT, 
                           PDF_LINK TEXT) """)
    con.commit()
    # ПОДУМАТЬ, КАКАЯ ЕЩЕ ТАБЛИЧКА МОЖЕТ ПОНАДОБИТСЯ. МБ ТАБЛИЧКА ПО АНАЛИЗУ ДОКОВ И ТАМ ТОЖЕ КАКАЯ-ТО ИНФА БУДЕТ -- УТОЧНИТЬ
    
    
    con.close()
    

In [ ]:
create_tables() # создание табличек выдает ошибку если таблички уже существуют 

In [ ]:
def add_constraints_on_tables(connection  = con):
    cursor_obj = con.cursor()
    cursor_obj.execute(""" ALTER TABLE SEARCH_PARAMS ADD CONSTRAINT ..... """)
    
    con.commit()
    
    cursor_obj.execute(""" ALTER TABLE DOC_INFO ADD CONSTRAINT .... """)
    
    con.commit()
    
    con.close()
    
    # первичные ключи определить и еще мб какую-н сущность

In [ ]:
# нужен автоинкремент, чтобы при добавлении в таблицу взять след значение как первичный ключ

In [113]:
def download_docs( browser, ins_vals,  connection = con):
    cursor_obj = con.cursor()
    #browser = get_initial_page()
    n = len(ins_vals['ordinal_number'])
    for i in range(n):
        print("Загрузка документа {0} из {1}".format(i + 1, n ))
        browser.get(ins_vals['pdf_link'][i]) # скачиваем документ по ссылке 
        
        cursor_obj.execute("""
        INSERT INTO DOC_INFO(
                    ID_DOC, 
                    CASE_NUMBER, 
                    DATE_ON_SITE, -- LAST DATE OF UPDATING DOCUMENT ON SITE 
                    COURT_NAME, 
                    DOC_TITLE, 
                    PDF_LINK)  
        VALUES (?, ?, ?, ?, ?, ?)""", 
                ( ins_vals['ordinal_number'][i],
                  ins_vals['case_number'][i],
                  ins_vals['date_doc'][i],
                  ins_vals['court_name'][i],
                  ins_vals['document_name'][i],
                  ins_vals['pdf_link'][i]) )
        
        time.sleep(1)
        
        now = datetime.datetime.now()
        
        cursor_obj.execute("""
        INSERT INTO SEARCH_PARAMS(
                    ID_DOC,
                    SEARCH_DIRECTION, 
                    DATE_FROM, 
                    DATE_TO,  
                    DOWNLOAD_DATE, 
                    LOCATED_IN_DIRECTORY)
                    
        VALUES (?, ?, ?, ?, ?, ?)""", 
                ( ins_vals['ordinal_number'][i],
                  doc_text_input,
                  date_from,
                  date_to,
                  now.strftime("%d-%m-%Y %H:%M"),
                  download_directory) )
        
        
        time.sleep(random.uniform(5, 10))
        
        con.commit()
        
        browser.quit()
        
        # Заполнять еще таблицу 
    

In [115]:
download_docs( browser = browser, ins_vals = res_insert_values ,  connection = con  ) # протестировать! чтобы загружалось 

Загрузка документа 1 из 325
Загрузка документа 2 из 325
Загрузка документа 3 из 325
Загрузка документа 4 из 325
Загрузка документа 5 из 325
Загрузка документа 6 из 325
Загрузка документа 7 из 325
Загрузка документа 8 из 325
Загрузка документа 9 из 325
Загрузка документа 10 из 325
Загрузка документа 11 из 325
Загрузка документа 12 из 325
Загрузка документа 13 из 325


KeyboardInterrupt: 

In [ ]:
# Тезаурус дока и их анализ ! Что делать конкретно?